In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Added for quick visual checks

# Set pandas to display all columns so you can see your changes
pd.set_option('display.max_columns', None)

In [ ]:
# REPLACE these filenames with your actual file paths
df = pd.read_csv('FEG_Main_Dataset.csv') 
df_abs = pd.read_csv('ABS_Industry_Wages.csv') 

print(f"Main Dataset Shape: {df.shape}")
print(f"ABS Dataset Shape: {df_abs.shape}")

# Debug: Check the data types. 
# If 'IP Weekly Wage' says 'object', that is why your code failed earlier.
print("\n--- Data Types Check ---")
print(df[['IP Weekly Wage', 'Industry']].dtypes)

In [ ]:
# 1. Clean Main Dataset 'IP Weekly Wage'
# Convert to string -> remove '$' and ',' -> Convert to Numeric
df['IP Weekly Wage'] = df['IP Weekly Wage'].astype(str).str.replace(r'[$,]', '', regex=True)
df['IP Weekly Wage'] = pd.to_numeric(df['IP Weekly Wage'], errors='coerce')

# 2. Clean ABS Dataset Wage Column
# We do the same thing here to ensure we can divide them later
if 'ABS_Average_Weekly_Wage' in df_abs.columns:
    df_abs['ABS_Average_Weekly_Wage'] = df_abs['ABS_Average_Weekly_Wage'].astype(str).str.replace(r'[$,]', '', regex=True)
    df_abs['ABS_Average_Weekly_Wage'] = pd.to_numeric(df_abs['ABS_Average_Weekly_Wage'], errors='coerce')

# Sanity Check: Did it work?
print("New Data Type for IP Weekly Wage:", df['IP Weekly Wage'].dtype)
print("Number of NaN wages (unparseable):", df['IP Weekly Wage'].isna().sum())

In [ ]:
# Define the target columns
target_cols = ['Annual Leave Reliability', 'Long Service Leave Reliability', 'Wages Reliability']

print("--- Cleaning Targets ---")
for col in target_cols:
    if col in df.columns:
        # 1. String Cleaning: Force to string, remove '%' and whitespace
        # This turns " 95% " into "95"
        df[col] = df[col].astype(str).str.replace('%', '', regex=False).str.strip()
        
        # 2. Convert to Numeric
        # 'errors=coerce' turns garbage (like "N/A" or "Error") into proper NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # 3. Scale Correction (Crucial Check)
        # If the data was "85.5", it is now 85.5. But we want 0.0 to 1.0.
        # Logic: If the max value is > 1.0, it implies 0-100 scale, so we divide by 100.
        if df[col].max() > 1.0:
            print(f"Detected 0-100 scale for {col}. Dividing by 100...")
            df[col] = df[col] / 100.0
            
        # 4. Clip values to ensure strict 0.0 - 1.0 range
        # (Handles negatives by turning them to 0.0)
        df[col] = df[col].clip(lower=0.0, upper=1.0)
        
        # 5. Drop rows where the Target is still NaN (cannot train on them)
        initial_len = len(df)
        df = df.dropna(subset=[col])
        dropped = initial_len - len(df)
        if dropped > 0:
            print(f"Dropped {dropped} rows with invalid/missing {col}")

# Final Sanity Check
print("\n--- Final Target Stats (Should be 0.0 to 1.0) ---")
print(df[target_cols].describe())

In [ ]:
# Cell 5: Internal Feature Engineering (Tenure) - CORRECTED

# 1. Convert to datetime with Australian format (dayfirst=True)
# 'errors=coerce' handles garbage text, 'dayfirst=True' handles the 13/01/2023 issue
df['IP Commencement Date'] = pd.to_datetime(df['IP Commencement Date'], dayfirst=True, errors='coerce')
df['IP Termination Date'] = pd.to_datetime(df['IP Termination Date'], dayfirst=True, errors='coerce')

# 2. Calculate Tenure in Years
df['IP_Tenure_Years'] = (df['IP Termination Date'] - df['IP Commencement Date']).dt.days / 365.25

# 3. Fix Logic: Negative tenure -> 0
df.loc[df['IP_Tenure_Years'] < 0, 'IP_Tenure_Years'] = 0

# 4. Fix Missing: Fill NaN with -1
df['IP_Tenure_Years'] = df['IP_Tenure_Years'].fillna(-1)

# Sanity Check: Look at the distribution
print(df['IP_Tenure_Years'].describe())

In [ ]:
if 'Industry' in df.columns:
    # Merge
    df = df.merge(df_abs[['Industry', 'ABS_Average_Weekly_Wage']], on='Industry', how='left')
    
    # Fill missing ABS wages with National Median
    national_median = df_abs['ABS_Average_Weekly_Wage'].median()
    df['ABS_Average_Weekly_Wage'] = df['ABS_Average_Weekly_Wage'].fillna(national_median)
    
    # Create Ratio Feature (Float divided by Float)
    df['IP_Wage_to_ABS_Ratio'] = df['IP Weekly Wage'] / (df['ABS_Average_Weekly_Wage'] + 1e-5)
    
    # Create High Risk Flag
    df['Flag_High_Wage_Deviation'] = (df['IP_Wage_to_ABS_Ratio'] > 1.5).astype(int)
    
    # Drop the raw dollar value
    df.drop(columns=['ABS_Average_Weekly_Wage'], inplace=True)

# Sanity Check: Show the new columns
print(df[['Industry', 'IP Weekly Wage', 'IP_Wage_to_ABS_Ratio']].head())

In [ ]:
cols_to_drop = [
    # Claimant Data
    'Claim ID', 'Claim Type', 'Claim Form Received Date', 'Claimant Age', 
    'Service Years At Appointment', 'Job Title', 'Job Duty Description', 
    'Claimant Confident of Amounts Owed', 'Information Held About Owed Entitlements',
    'Claimant Commencement Date', 'Claimant Termination Date', 
    'Claimant Weekly Wage', 'Claimant Annual Leave', 'Claimant Wages',
    
    # CM Recommended (Leakage)
    'CM Recommended Employment Type', 'CM Recommended Weekly Wage', 
    'CM Recommended Commencement Date', 'CM Recommended Termination Date',
    'CM Recommended Annual Leave', 'CM Recommended Long Service Leave', 
    'CM Recommended Wages',
    
    # Raw IP Dates (We have Tenure now)
    'IP Commencement Date', 'IP Termination Date'
]

# Only drop cols that exist
existing_cols = [c for c in cols_to_drop if c in df.columns]
df.drop(columns=existing_cols, inplace=True)

# Sanity Check: Check if any 'CM Recommended' columns remain
leaking = [c for c in df.columns if 'CM Recommended' in c]
print("Leaking columns remaining (Should be empty):", leaking)

In [ ]:
# Cell 8: Final Cleanup (Corrected Order)

days_col = 'Days Between IP Verified Data Request and Received Date'

if days_col in df.columns:
    print(f"Rows before cleanup: {len(df)}")
    
    # 1. Handle Negatives (Don't drop the row, just mark the value as NaN)
    # Logic: Negative time is impossible, so it's a data error.
    mask_negatives = df[days_col] < 0
    print(f"Found {mask_negatives.sum()} negative values. Setting them to NaN.")
    df.loc[mask_negatives, days_col] = np.nan
    
    # 2. Impute Missing Values (Crucial Step to save the 5,000 rows)
    # We calculate the median based on the valid positive numbers
    median_days = df[days_col].median()
    print(f"Imputing missing days with Median: {median_days}")
    
    # Add a "Was Missing" flag so the model knows this was imputed
    df[f'{days_col}_missing'] = df[days_col].isna().astype(int)
    df[days_col] = df[days_col].fillna(median_days)
    
    # 3. Winsorize (Cap extreme highs)
    # Now that we have full data, we cap the top 1%
    limit = df[days_col].quantile(0.99)
    print(f"Capping extreme values at: {limit}")
    df.loc[df[days_col] > limit, days_col] = limit

# 4. General Imputation for all other columns
for col in df.columns:
    if df[col].isna().sum() > 0:
        # Create indicator
        df[f'{col}_missing'] = df[col].isna().astype(int)
        
        # Fill values
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].median())
        else:
            mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
            df[col] = df[col].fillna(mode_val)

print(f"Final Data Shape: {df.shape}")
print("Cleaning Complete.")

In [ ]:
# Cell 9: Service Provider Reputation Feature

# 1. Define the ID and the Target we want to use as a proxy for "Reputation"
provider_col = 'Service Provider Id'
target_col = 'Wages Reliability' # Using Wages as the general signal for reliability

if provider_col in df.columns and target_col in df.columns:
    print("--- Engineering Service Provider Reputation ---")
    
    # Ensure ID is treated as a category, not a number
    df[provider_col] = df[provider_col].astype(str)

    # Calculate Global Mean (Baseline for new/unknown providers)
    global_mean = df[target_col].mean()
    
    # Calculate Sum and Count per Provider
    grouped = df.groupby(provider_col)[target_col]
    provider_sum = grouped.transform('sum')
    provider_count = grouped.transform('count')
    
    # --- LEAVE-ONE-OUT CALCULATION ---
    # Formula: (Sum of all their scores - Current Row's Score) / (Total Count - 1)
    # This ensures the model can't "cheat" by seeing the answer in the input
    df['Provider_Reputation_Score'] = (provider_sum - df[target_col]) / (provider_count - 1)
    
    # Handle Edge Cases:
    # 1. If a provider has only 1 case, (Count - 1) is 0 -> Division by Zero -> NaN/Inf
    # 2. We fill these gaps with the Global Mean (innocent until proven guilty)
    df['Provider_Reputation_Score'] = df['Provider_Reputation_Score'].fillna(global_mean)
    # Also handle explicit infinity if numpy generated it
    df['Provider_Reputation_Score'] = df['Provider_Reputation_Score'].replace([np.inf, -np.inf], global_mean)
    
    # Drop the raw ID columns (The model uses the score now)
    df.drop(columns=[provider_col, 'Service Provider Contact Id'], inplace=True, errors='ignore')
    
    print(f"Feature Created: 'Provider_Reputation_Score'")
    print(f"Global Mean Reliability: {global_mean:.4f}")
    print(df[['Provider_Reputation_Score', target_col]].head())

else:
    print(f"Skipping: {provider_col} or {target_col} not found.")

In [ ]:
# Cell 10: Final Pre-Flight Check

print("--- Final Dataset Status ---")

# 1. Check for Non-Numeric Columns
# (Should only be empty or maybe 'Industry' if you want to One-Hot Encode it later)
non_numerics = df.select_dtypes(include=['object']).columns.tolist()

if len(non_numerics) > 0:
    print(f"⚠️ WARNING: The following columns are still text: {non_numerics}")
    print("Action: You will need to One-Hot Encode these (e.g. pd.get_dummies) before training.")
else:
    print("✅ SUCCESS: All columns are numeric.")

# 2. Check for Missing Values
nan_counts = df.isna().sum().sum()
if nan_counts > 0:
    print(f"❌ CRITICAL: {nan_counts} missing values remain.")
    # Show which columns have them
    print(df.columns[df.isna().any()].tolist())
else:
    print("✅ SUCCESS: No missing values found.")

# 3. Check Feature Power (Correlation)
# Does our new 'Reputation' feature actually work?
if 'Provider_Reputation_Score' in df.columns and 'Wages Reliability' in df.columns:
    corr = df['Provider_Reputation_Score'].corr(df['Wages Reliability'])
    print(f"\nCorrelation between Provider Reputation & Reliability: {corr:.4f}")
    if corr > 0.1:
        print("✅ STRONG SIGNAL: Past behavior predicts future reliability.")
    elif corr < 0.05:
        print("⚠️ WEAK SIGNAL: Provider history might not be predictive in this dataset.")

print(f"\nFinal Shape for Training: {df.shape}")